In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [11]:
cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code', 'Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10', 'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15']

nppes_chunks = pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols=cols, iterator=True, chunksize=100, nrows = 1000)

In [12]:
var_cols = ['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch']


In [13]:
state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.DataFrame()
id_cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code']

for chunk in nppes_chunks:
    filtered_chunk = chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)]
    filtered_chunk = pd.wide_to_long(filtered_chunk, var_cols, i = id_cols , j = "", sep = "_")
    
    nppes = pd.concat([nppes, filtered_chunk])
    

In [3]:
state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.concat([chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)] for chunk in nppes_chunks])

In [16]:
nppes.reset_index()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,,Healthcare Provider Taxonomy Code,Healthcare Provider Primary Taxonomy Switch
0,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,1,183500000X,Y
1,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,2,NaN,NaN
2,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,3,NaN,NaN
3,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,4,NaN,NaN
4,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,1477556611,1.0,NaN,LEWIS,ANESSA,J,NaN,NaN,M.D.,2950 VILLAGE DR,NaN,FAYETTEVILLE,NC,283043815.0,11,NaN,NaN
536,1477556611,1.0,NaN,LEWIS,ANESSA,J,NaN,NaN,M.D.,2950 VILLAGE DR,NaN,FAYETTEVILLE,NC,283043815.0,12,NaN,NaN
537,1477556611,1.0,NaN,LEWIS,ANESSA,J,NaN,NaN,M.D.,2950 VILLAGE DR,NaN,FAYETTEVILLE,NC,283043815.0,13,NaN,NaN
538,1477556611,1.0,NaN,LEWIS,ANESSA,J,NaN,NaN,M.D.,2950 VILLAGE DR,NaN,FAYETTEVILLE,NC,283043815.0,14,NaN,NaN


In [ ]:
hop_ex = pd.read_csv('DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', 
                          nrows = 100)

hop_ex.head()

In [ ]:
chunks = pd.read_csv('DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
hop = []
for chunk in chunks: 
    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
    hop.append(chunk)

In [ ]:
hop = pd.concat(hop, ignore_index = True)

hop

In [ ]:
db = sqlite3.connect('DocGraph_Hop_Teaming_2018_Commercial/hop.sqlite')

chunks = pd.read_csv('DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
hop = []
for chunk in chunks:
    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
    chunk.to_sql('hop', db, if_exists = 'append', index = False)            # Append the chunk to a calls table